# Load Depenency

In [1]:
#Cleaned up Crypto Example
from os import system
import pandas as pd
import requests
import datetime as dt
# from pytrials.client import ClinicalTrials
import json
import ipywidgets as widgets
import numpy as np
np.random.seed(10031975)
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rcParams, cycler
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator

In [2]:
#does some error handling


import sys
import csv
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

In [3]:
from pytrials.utils import json_handler, csv_handler


class ClinicalTrials:
    """ClinicalTrials API client

    Provides functions to easily access the ClinicalTrials.gov API
    (https://clinicaltrials.gov/api/)
    in Python.

    Attributes:
        study_fields: List of all study fields you can use in your query.
        api_info: Tuple containing the API version number and the last
        time the database was updated.
    """

    _BASE_URL = "https://clinicaltrials.gov/api/"
    _INFO = "info/"
    _QUERY = "query/"
    _JSON = "fmt=json"
    _CSV = "fmt=csv"

    def __init__(self):
        self.api_info = self.__api_info()

    @property
    def study_fields(self):
        fields_list = json_handler(
            f"{self._BASE_URL}{self._INFO}study_fields_list?{self._JSON}"
        )
        return fields_list["StudyFields"]["Fields"]

    def __api_info(self):
        """Returns information about the API"""
        last_updated = json_handler(
            f"{self._BASE_URL}{self._INFO}data_vrs?{self._JSON}"
        )["DataVrs"]
        api_version = json_handler(f"{self._BASE_URL}{self._INFO}api_vrs?{self._JSON}")[
            "APIVrs"
        ]

        return api_version, last_updated

    def get_full_studies(self, search_expr, max_studies=50):
        """Returns all content for a maximum of 100 study records.

        Retrieves information from the full studies endpoint, which gets all study fields.
        This endpoint can only output JSON (Or not-supported XML) format and does not allow
        requests for more than 100 studies at once.

        Args:
            search_expr (str): A string containing a search expression as specified by
                `their documentation <https://clinicaltrials.gov/api/gui/ref/syntax#searchExpr>`_.
            max_studies (int): An integer indicating the maximum number of studies to return.
                Defaults to 50.

        Returns:
            dict: Object containing the information queried with the search expression.

        Raises:
            ValueError: The number of studies can only be between 1 and 100
        """
        if max_studies > 100 or max_studies < 1:
            raise ValueError("The number of studies can only be between 1 and 100")

        req = f"full_studies?expr={search_expr}&max_rnk={max_studies}&{self._JSON}"

        full_studies = json_handler(f"{self._BASE_URL}{self._QUERY}{req}")

        return full_studies

    def get_study_fields(self, search_expr, fields, max_studies=50, min_rnk=1,fmt="csv"):
        """Returns study content for specified fields

        Retrieves information from the study fields endpoint, which acquires specified information
        from a large (max 1000) studies. To see a list of all possible fields, check the class'
        study_fields attribute.

        Args:
            search_expr (str): A string containing a search expression as specified by
                `their documentation <https://clinicaltrials.gov/api/gui/ref/syntax#searchExpr>`_.
            fields (list(str)): A list containing the desired information fields.
            max_studies (int): An integer indicating the maximum number of studies to return.
                Defaults to 50.
            fmt (str): A string indicating the output format, csv or json. Defaults to csv.

        Returns:
            Either a dict, if fmt='json', or a list of records (e.g. a list of lists), if fmt='csv.
            Both containing the maximum number of study fields queried using the specified search expression.

        Raises:
            ValueError: The number of studies can only be between 1 and 1000
            ValueError: One of the fields is not valid! Check the study_fields attribute
                for a list of valid ones.
            ValueError: Format argument has to be either 'csv' or 'json'
        """
        if max_studies > 1000 or max_studies < 1:
            raise ValueError("The number of studies can only be between 1 and 1000")
        elif not set(fields).issubset(self.study_fields):
            raise ValueError(
                "One of the fields is not valid! Check the study_fields attribute for a list of valid ones."
            )
        else:
            concat_fields = ",".join(fields)
            # req = f"study_fields?expr={search_expr}&max_rnk={max_studies}&fields={concat_fields}"
            req = f"study_fields?expr={search_expr}&min_rnk={min_rnk}&max_rnk={max_studies+min_rnk-1}&fields={concat_fields}"
            if fmt == "csv":
                url = f"{self._BASE_URL}{self._QUERY}{req}&{self._CSV}"
                return csv_handler(url)

            elif fmt == "json":
                url = f"{self._BASE_URL}{self._QUERY}{req}&{self._JSON}"
                return json_handler(url)

            else:
                raise ValueError("Format argument has to be either 'csv' or 'json'")

    def get_study_count(self, search_expr):
        """Returns study count for specified search expression

        Retrieves the count of studies matching the text entered in search_expr.

        Args:
            search_expr (str): A string containing a search expression as specified by
                `their documentation <https://clinicaltrials.gov/api/gui/ref/syntax#searchExpr>`_.

        Returns:
            An integer

        Raises:
            ValueError: The search expression cannot be blank.
        """
        if not set(search_expr):
            raise ValueError("The search expression cannot be blank.")
        else:
            req = f"study_fields?expr={search_expr}&max_rnk=1&fields=NCTId"
            url = f"{self._BASE_URL}{self._QUERY}{req}&{self._JSON}"
            returned_data = json_handler(url)
            study_count = returned_data["StudyFieldsResponse"]["NStudiesFound"]
            return study_count

    def __repr__(self):
        return f"ClinicalTrials.gov client v{self.api_info[0]}, database last updated {self.api_info[1]}"

# Load Data

https://clinicaltrials.gov/api/gui/ref/crosswalks

Disease 2 search for

This is for dymanic input of diease searching

In [4]:
disease=input()
print(disease)

parkinson


In [5]:
# disease = 'parkinson'

#disease = 'depression'

# disease = 'diabetes'

# disease = 'tacs'



hit mesh some day

Pull ONE

#Explore Data

In [6]:
#change this out to NAME

ct = ClinicalTrials()
infodf= (ct.get_full_studies(search_expr=disease, max_studies=5 ))


In [7]:
type(infodf)

dict

In [8]:
number = print(infodf['FullStudiesResponse']['NStudiesFound'])

3662


In [9]:
count = ct.get_study_count(search_expr=disease)


In [10]:
# count = 3000
start = 0
end = count
iter = 999 #number of studies to get at a time ( limited to 1000 )





temp = pd.DataFrame()
temp2 = pd.DataFrame()
new_df = pd.DataFrame()


for i in range(1, count, iter):
    # print(temp)
    temp = ct.get_study_fields(
        search_expr=disease,
        fields=["NCTId", "Condition", "OfficialTitle", "BriefTitle" , "Acronym" , "StudyType",
    "InterventionType","InterventionName","InterventionOtherName","InterventionDescription","Phase" ,"StudyFirstSubmitDate","LastUpdateSubmitDate","CompletionDate","OverallStatus","BaselineMeasureTitle","BaselineMeasureDescription","BaselineMeasurementValue","BriefSummary"],
        max_studies=iter,
        min_rnk=i,
        fmt="csv",)
    # print(temp2[0])
    temp2 =  pd.DataFrame.from_records(temp[1:])
    new_df = pd.concat([new_df, temp2])
 


In [11]:
new_df.columns = ["Index","NCTId", "Condition", "OfficialTitle", "BriefTitle" , "Acronym" , "StudyType",
    "InterventionType","InterventionName","InterventionOtherName","InterventionDescription","Phase" ,"StudyFirstSubmitDate","LastUpdateSubmitDate","CompletionDate","OverallStatus","BaselineMeasureTitle","BaselineMeasureDescription","BaselineMeasurementValue","BriefSummary"]

In [12]:
# new_df
new_df.shape

(3662, 20)

In [13]:
new_df.head

<bound method NDFrame.head of     Index        NCTId                            Condition  \
0       1  NCT02370134                  Parkinson's Disease   
1       2  NCT01662427                  Parkinson's Disease   
2       3  NCT05245955                    Parkinson Disease   
3       4  NCT03152721                    Parkinson Disease   
4       5  NCT02016092                  Parkinson's Disease   
..    ...          ...                                  ...   
660  3658  NCT03681509  Emotions|Motivation|Reward|Dopamine   
661  3659  NCT02704520                        Rectal Cancer   
662  3660  NCT04370470                   Huntington Disease   
663  3661  NCT05168059                  High Blood Pressure   
664  3662  NCT02069509                  Friedreich's Ataxia   

                                         OfficialTitle  \
0    Development of Parkinson's Glove for Detection...   
1    Nocturnal Parkinson's Disease Symptoms Study G...   
2    Feasibility and Validity of Roboti

In [14]:
# temp = pd.DataFrame.from_records(api_pull_2[1:], columns=api_pull_2[0])

In [15]:
# print(temp.shape)

In [16]:
# count = 3000
start = 0
end = count
iter = 999 #number of studies to get at a time ( limited to 1000 )

from random import seed
from random import random



temp = pd.DataFrame()
temp2 = pd.DataFrame()
new_df2 = pd.DataFrame()


for i in range(1, count, iter):
    # print(temp)
    temp = ct.get_study_fields(
        search_expr=disease,
        fields=["NCTId","IsFDARegulatedDrug","IsFDARegulatedDevice", "IsUnapprovedDevice", "PrimaryOutcomeMeasure", "PrimaryOutcomeDescription","PrimaryOutcomeTimeFrame", "SecondaryOutcomeMeasure","SecondaryOutcomeDescription", "SecondaryOutcomeTimeFrame","OtherOutcomeMeasure"
,"OtherOutcomeDescription","OtherOutcomeTimeFrame","EligibilityCriteria","StudyPopulation","HealthyVolunteers", "ReferencePMID", "LocationCity", "LocationState" , "LocationFacility"],
        max_studies=iter,
        min_rnk=i,
        fmt="csv",)
    # print(temp2[0])
    temp2 =  pd.DataFrame.from_records(temp[1:])
    new_df2 = pd.concat([new_df2, temp2])
 


In [17]:
new_df2.columns = ["index", "NCTId","IsFDARegulatedDrug","IsFDARegulatedDevice", "IsUnapprovedDevice", "PrimaryOutcomeMeasure", "PrimaryOutcomeDescription","PrimaryOutcomeTimeFrame", "SecondaryOutcomeMeasure","SecondaryOutcomeDescription", "SecondaryOutcomeTimeFrame","OtherOutcomeMeasure"
,"OtherOutcomeDescription","OtherOutcomeTimeFrame","EligibilityCriteria","StudyPopulation","HealthyVolunteers", "ReferencePMID", "LocationCity", "LocationState" , "LocationFacility"]

In [18]:
new_df2.shape

(3662, 21)

In [19]:
new_df2

,index,NCTId,IsFDARegulatedDrug,IsFDARegulatedDevice,IsUnapprovedDevice,PrimaryOutcomeMeasure,PrimaryOutcomeDescription,PrimaryOutcomeTimeFrame,SecondaryOutcomeMeasure,SecondaryOutcomeDescription,...,OtherOutcomeMeasure,OtherOutcomeDescription,OtherOutcomeTimeFrame,EligibilityCriteria,StudyPopulation,HealthyVolunteers,ReferencePMID,LocationCity,LocationState,LocationFacility
0,1,NCT02370134,,,,Number of participants who demonstrate the tre...,To determine the tremor suppression after use ...,up to 1 month follow up period,Number of Participants who report pain or any ...,To determine the safety of parkinson's glove|T...,...,,,,Inclusion Criteria:||• Adults ≥ 18 years old.|...,,No,,Pathumwan,Bangkok,Chulalongkorn University Hospital
1,2,NCT01662427,,,,To assess Noctural Parkinson's Disease Symptoms,,,To assess quality of life in Parkinson's Disea...,,...,,,,Inclusion Criteria:||Idiopathic Parkinson's di...,Parkinson's disease patient and care givers,Accepts Healthy Volunteers,,Pathumwan,Bangkok,Roongroj Bhidayasiri
2,3,NCT05245955,No,No,,Detection threshold resulting from the assessm...,Outcome of the assessment for the detection of...,"Through study completion, an average of 2 week...",Demographic information|Edinburgh Handedness I...,"Gender, age, year of birth.|Questionnaire to d...",...,,,,Ten patients with Parkinson's disease and 10 a...,,Accepts Healthy Volunteers,,,,
3,4,NCT03152721,No,No,,Decision to change management,The decision to change management of Parkinson...,"On first regular contact, RC, (telephone or vi...",Actual change of management|Patient Reported E...,A retrospectively verified (medical and prescr...,...,,,,Inclusion Criteria:||Clinical diagnosis of Par...,,No,,Göteborg,,Sahlgrenska University Hospital
4,5,NCT02016092,,,,Biological markers of Parkinson's disease,To identify biologic markers of Parkinson's Di...,18 months,Correlation between biologic markers and clini...,To identify and investigate possible correlati...,...,,,,Inclusion Criteria:||For PD cases:||Inclusion ...,Parkinson's patients,Accepts Healthy Volunteers,,Sunnyvale,California,The Parkinson's Institute and Clinical Center
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
660,3658,NCT03681509,No,No,,Change in reward sensitivity on a reinforcemen...,Participants are shown two distinct abstract s...,Completed at day 12 to 15 of treatment,Change in performance in a facial expression r...,Participants are presented with individual pic...,...,Change in subjective mood and energy|Change in...,Zersen's Befindlichkeits-Skala|Positive and Ne...,Completed at day 12 to 15 of treatment|Complet...,Inclusion Criteria:||Male or female|Age: 18 to...,,Accepts Healthy Volunteers,,Oxford,Oxfordshire,"Dept of Psychiatry, University of Oxford"
661,3659,NCT02704520,,,,Patient recruitment rate,Total number of patients randomised per month ...,During last 4 months of recruitment,pathology circumferential resection margin (pC...,Overall pCRM involvement rate between the cont...,...,,,,Inclusion Criteria:||Have a biopsy-confirmed a...,,No,28851403,Aberdeen|Basingstoke|Stoke-on-Trent|Salisbury|...,Aberdeenshire|Hampshire|Staffordshire|Wiltshire,Aberdeen Royal Infirmary - NHS Grampion|Hampsh...
662,3660,NCT04370470,No,No,,The UHDRS SIF ratings|Clinimetric properties o...,The equivalence of the original UHDRS function...,3 weeks|3 weeks,,,...,,,,Inclusion Criteria:||Participants must meet al...,Manifest HDGEC Participants: Manifest HDGEC Pa...,No,,Birmingham|La Jolla|Los Angeles|Englewood|Chic...,Alabama|California|California|Colorado|Illinoi...,The Board of Trustees of the University of Ala...
663,3661,NCT05168059,No,No,,Effectiveness of clevidipine,Percentage of patients achieving target systol...,1 to 6 hour of clevidipine infusion beginning,Incidence of adverse events related to clevidi...,"Tachycardia, atrial fibrillation, hypotension,...",...,Effectiveness of clevidipine treatment accordi...,Percentage of male and female pa

In [20]:




# api_pull_2 = ct.get_study_fields(
#     search_expr="Parkinson",
#     fields=["NCTId"],
#     max_studies=10,
#     min_rnk=5,
#     fmt="csv",
# )

# # ClinicalTrials limits API queries to 1000 records
# # Count of studies may be useful to build loops when you want to retrieve more than 1000 records



# # Read the csv data in Pandas
# import pandas as pd

# temp = pd.DataFrame.from_records(api_pull_2[1:], columns=api_pull_2[0])
# temp

In [21]:




# # Get the NCTId, Condition and Brief title fields from 500 studies related to Coronavirus and Covid, in csv format.
# api_pull_1 = ct.get_study_fields(
#     search_expr="Parkinson",
#     fields=["NCTId", "Condition", "OfficialTitle", "BriefTitle" , "Acronym" , "StudyType",
#     "InterventionType","InterventionName","InterventionOtherName","InterventionDescription","Phase" 
#     ,"StudyFirstSubmitDate","LastUpdateSubmitDate","CompletionDate","OverallStatus","IsFDARegulatedDrug","IsFDARegulatedDevice","BriefSummary"],
#     max_studies=999,
#     fmt="csv",
# )

# # ClinicalTrials limits API queries to 1000 records
# # Count of studies may be useful to build loops when you want to retrieve more than 1000 records



# # Read the csv data in Pandas
# import pandas as pd

# df1=pd.DataFrame.from_records(api_pull_1[1:], columns=api_pull_1[0])

# # 
# # df1 = pd.DataFrame.to_frame().reset_index()



In [22]:
# api_pull_2 = ct.get_study_fields(
#     search_expr="Parkinson",
#     fields=["NCTId","IsFDARegulatedDrug","IsFDARegulatedDevice", "IsUnapprovedDevice", "PrimaryOutcomeMeasure", "PrimaryOutcomeDescription","PrimaryOutcomeTimeFrame", "SecondaryOutcomeMeasure","SecondaryOutcomeDescription", "SecondaryOutcomeTimeFrame","OtherOutcomeMeasure"
# ,"OtherOutcomeDescription","OtherOutcomeTimeFrame","EligibilityCriteria","StudyPopulation","HealthyVolunteers", "ReferencePMID", "LocationCity", "LocationState" , "LocationFacility"

# ],
#     max_studies=999,
#     fmt="csv",
# )

# # ClinicalTrials limits API queries to 1000 records
# # Count of studies may be useful to build loops when you want to retrieve more than 1000 records

# ct.get_study_count(search_expr="Parkinson")

# # Read the csv data in Pandas
# import pandas as pd

# df2 = pd.DataFrame.from_records(api_pull_2[1:], columns=api_pull_2[0])

##cleaning

In [23]:
# print(type(df1))

In [24]:
result = pd.merge(new_df, new_df2,  on='NCTId', how='outer')

In [25]:
result.shape

(3662, 40)

In [26]:
# result = result.loc[:,~result.columns.duplicated()]

In [27]:
# result.shape

In [28]:
# result.columns.tolist()

IMPORTANT - this just drops the second index column - monitor or watch for mismatch or if you change the number of elements that are pulled from the first api

In [29]:
result = result.drop(result.columns[[20]], axis=1)

In [30]:
result.shape

(3662, 39)

In [31]:
# result.columns.tolist()

Save file

In [32]:
# result.to_csv(r'data_raw.csv', index = False)
result.to_excel(r'data_raw.xlsx', index = False)

In [33]:
result.to_hdf('data_raw.h5', key='df', mode='w')  

In [34]:
# labels, levels = pd.factorize(result["Phase"])  

In [35]:
# labels

In [36]:
# fun_barplot()